
<a id='sparse'></a>
{{ header }}

# Sparse data structures

>**Note**
>
>`SparseSeries` and `SparseDataFrame` have been deprecated. Their purpose
is served equally well by a `Series` or `DataFrame` with
sparse values. See [Migrating](#sparse-migration) for tips on migrating.

Pandas provides data structures for efficiently storing sparse data.
These are not necessarily sparse in the typical “mostly 0”. Rather, you can view these
objects as being “compressed” where any data matching a specific value (`NaN` / missing value, though any value
can be chosen, including 0) is omitted. The compressed values are not actually stored in the array.

Notice the dtype, `Sparse[float64, nan]`. The `nan` means that elements in the
array that are `nan` aren’t actually stored, only the non-`nan` elements are.
Those non-`nan` elements have a `float64` dtype.

The sparse objects exist for memory efficiency reasons. Suppose you had a
large, mostly NA `DataFrame`:

As you can see, the density (% of values that have not been “compressed”) is
extremely low. This sparse object takes up much less memory on disk (pickled)
and in the Python interpreter.

Functionally, their behavior should be nearly
identical to their dense counterparts.


<a id='sparse-array'></a>

## SparseArray

`SparseArray` is a `ExtensionArray`
for storing an array of sparse values (see [dtypes](getting_started/basics.ipynb#basics-dtypes) for more
on extension arrays). It is a 1-dimensional ndarray-like object storing
only values distinct from the `fill_value`:

A sparse array can be converted to a regular (dense) ndarray with `numpy.asarray()`


<a id='sparse-dtype'></a>

## SparseDtype

The `SparseArray.dtype` property stores two pieces of information

1. The dtype of the non-sparse values  
1. The scalar fill value  


A `SparseDtype` may be constructed by passing each of these

The default fill value for a given NumPy dtype is the “missing” value for that dtype,
though it may be overridden.

Finally, the string alias `'Sparse[dtype]'` may be used to specify a sparse dtype
in many places


<a id='sparse-accessor'></a>

## Sparse accessor

New in version 0.24.0.

Pandas provides a `.sparse` accessor, similar to `.str` for string data, `.cat`
for categorical data, and `.dt` for datetime-like data. This namespace provides
attributes and methods that are specific to sparse data.

This accessor is available only on data with `SparseDtype`, and on the `Series`
class itself for creating a Series with sparse data from a scipy COO matrix with.

New in version 0.25.0.

A `.sparse` accessor has been added for `DataFrame` as well.
See api.frame.sparse for more.


<a id='sparse-calculation'></a>

## Sparse calculation

You can apply NumPy [ufuncs](https://docs.scipy.org/doc/numpy/reference/ufuncs.html)
to `SparseArray` and get a `SparseArray` as a result.

The *ufunc* is also applied to `fill_value`. This is needed to get
the correct dense result.


<a id='sparse-migration'></a>

## Migrating

In older versions of pandas, the `SparseSeries` and `SparseDataFrame` classes (documented below)
were the preferred way to work with sparse data. With the advent of extension arrays, these subclasses
are no longer needed. Their purpose is better served by using a regular Series or DataFrame with
sparse values instead.

>**Note**
>
>There’s no performance or memory penalty to using a Series or DataFrame with sparse values,
rather than a SparseSeries or SparseDataFrame.

This section provides some guidance on migrating your code to the new style. As a reminder,
you can use the python warnings module to control warnings. But we recommend modifying
your code, rather than ignoring the warning.

**Construction**

From an array-like, use the regular `Series` or
`DataFrame` constructors with `SparseArray` values.

In [ ]:
# Previous way
>>> pd.SparseDataFrame({"A": [0, 1]})

From a SciPy sparse matrix, use `DataFrame.sparse.from_spmatrix()`,

In [ ]:
# Previous way
>>> from scipy import sparse
>>> mat = sparse.eye(3)
>>> df = pd.SparseDataFrame(mat, columns=['A', 'B', 'C'])

**Conversion**

From sparse to dense, use the `.sparse` accessors

From dense to sparse, use `DataFrame.astype()` with a `SparseDtype`.

**Sparse Properties**

Sparse-specific properties, like `density`, are available on the `.sparse` accessor.

**General differences**

In a `SparseDataFrame`, *all* columns were sparse. A `DataFrame` can have a mixture of
sparse and dense columns. As a consequence, assigning new columns to a `DataFrame` with sparse
values will not automatically convert the input to be sparse.

In [ ]:
# Previous Way
>>> df = pd.SparseDataFrame({"A": [0, 1]})
>>> df['B'] = [0, 0]  # implicitly becomes Sparse
>>> df['B'].dtype
Sparse[int64, nan]

Instead, you’ll need to ensure that the values being assigned are sparse

The `SparseDataFrame.default_kind` and `SparseDataFrame.default_fill_value` attributes
have no replacement.


<a id='sparse-scipysparse'></a>

## Interaction with scipy.sparse

Use `DataFrame.sparse.from_spmatrix()` to create a `DataFrame` with sparse values from a sparse matrix.

New in version 0.25.0.

All sparse formats are supported, but matrices that are not in [`COOrdinate`](https://docs.scipy.org/doc/scipy/reference/sparse.html#module-scipy.sparse) format will be converted, copying data as needed.
To convert back to sparse SciPy matrix in COO format, you can use the `DataFrame.sparse.to_coo()` method:

meth:Series.sparse.to_coo is implemented for transforming a `Series` with sparse values indexed by a `MultiIndex` to a [`scipy.sparse.coo_matrix`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.coo_matrix.html#scipy.sparse.coo_matrix).

The method requires a `MultiIndex` with two or more levels.

In the example below, we transform the `Series` to a sparse representation of a 2-d array by specifying that the first and second `MultiIndex` levels define labels for the rows and the third and fourth levels define labels for the columns. We also specify that the column and row labels should be sorted in the final sparse representation.

Specifying different row and column labels (and not sorting them) yields a different sparse matrix:

A convenience method `Series.sparse.from_coo()` is implemented for creating a `Series` with sparse values from a `scipy.sparse.coo_matrix`.

The default behaviour (with `dense_index=False`) simply returns a `Series` containing
only the non-null entries.

Specifying `dense_index=True` will result in an index that is the Cartesian product of the
row and columns coordinates of the matrix. Note that this will consume a significant amount of memory
(relative to `dense_index=False`) if the sparse matrix is large (and sparse) enough.


<a id='sparse-subclasses'></a>

## Sparse subclasses

The `SparseSeries` and `SparseDataFrame` classes are deprecated. Visit their
API pages for usage.